In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import regex as re

In [2]:
data= open("../datasets/llm/shakeshpere.txt",encoding="utf-8", mode ="r").read()
len(data)
bytesData = [x for x in data.encode(errors="replace")]
x =ord("a")
len(bytesData)

1115394

In [35]:
class BPETokenizer:
    def __init__(self, nmerges: int):
        self.nextByte: int= 256
        self.nmerges: int = nmerges
        pass
    
    def _get_counts(self,byteData:list):
        counts = {}
        for i,j in zip(byteData,byteData[1:]):
            counts[i,j]=counts.get((i,j),0)+1
        return counts

    def _do_merges(self,keys:tuple,token:int,byteData: list) -> list:
        i=0
        while i < (len(byteData)-1):
            i+=1
            if byteData[i] == keys[0] and byteData[i+1] == keys[1]:
                byteData[i] = token
                byteData.pop(i+1)
                i+=1
        return byteData
    
    def train(self,text:str):
        self.bytesData = [x for x in text.encode(errors="replace")]
        self.merges_dict: dict = {}
        for x in range(self.nmerges):
            bp_counts = self._get_counts(self.bytesData)
            req_key = max(bp_counts.items(),key=lambda x:x[1])[0]
            self.merges_dict[req_key] = self.nextByte + x
            self.bytesData = self._do_merges(req_key,self.nextByte+x,self.bytesData)
        
        self.decode_dict = {v:k for k,v in self.merges_dict.items()}
        

    def encode(self,text:str) -> list:
        tokens = [x for x in text.encode(errors="replace")]
        i=0
        while i < (len(tokens)):
            
            if i < (len(tokens)-1) and self.merges_dict.get((tokens[i],tokens[i+1]),False):
                tokens[i] = self.merges_dict[(tokens[i],tokens[i+1])]
                tokens.pop(i+1)
                i+=1
            else:
                i+=1
        
        return tokens
                

    def decode(self,tokens:list) -> str:
        bytes = []
        i=0
        while i < (len(tokens)):
            
            if self.decode_dict.get(tokens[i],False):
                bytes.extend(self.decode_dict[tokens[i]])
                i+=1
            else:
                bytes.extend([tokens[i]])
                i+=1
        return "".join([chr(x) for x in bytes])

In [36]:
tokenizer = BPETokenizer(10)

In [37]:
tokenizer.train(data)

In [38]:
tokenizer.encode("Hello")

[72, 101, 108, 108, 111]

In [41]:
tokenizer.decode(tokenizer.encode(data[:100]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [42]:
data[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [21]:
ord("y")

121

In [34]:
tokenizer.merges_dict

{(101, 32): 256,
 (116, 104): 256,
 (116, 32): 256,
 (115, 32): 256,
 (100, 32): 256,
 (44, 32): 256,
 (111, 117): 256,
 (101, 114): 256,
 (105, 110): 256,
 (121, 32): 256}